In [1]:
import json
with open('./mw/Residence_Haeundae.json','r') as f:
    review_list = json.load(f) 

review_list[:3]

FileNotFoundError: [Errno 2] No such file or directory: './mw/Residence_Haeundae.json'

In [46]:
good_cnt, bad_cnt=0,0

for review in review_list: 
    if review['stars'] == 5: 
        good_cnt += 1
    else:
        bad_cnt += 1 

good_cnt, bad_cnt

(0, 420)

In [47]:
reviews_good, reviews_bad = [], []  

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]'+ review['review' ] +'[REVIEW_END]')
    else: 
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

reviews_bad[:3]
        

['[REVIEW_START]30층 오션뷰였는데 앉아있기만 해도 힐링 그 자체였습니다.\r\n1층까지 환승해서 가야되는 불편함 말고는 대만족입니다!\r\n출장으로 내려왔지만 다시오고싶습니다.[REVIEW_END]',
 '[REVIEW_START]뷰가 정말 좋아요!\r\n바로앞에 해운대바다고 1분거리에 해운대야시장\r\n위치적으로도 뷰도 최곱니다ㅋㅋㅋ\r\n그리고 직원분들도 너무 친절하세요!!!\r\n아빠모시고왔는데 만족해하셔서 기분이 좋네요~후기가 많이 갈려서 걱정했는데 청결했어요![REVIEW_END]',
 '[REVIEW_START]엘리베이터 환승이 불편했지만\n객실내 깔끔하게 정리가 잘 되어 있고\n전망이 나쁘지 않아 좋았어요[REVIEW_END]']

In [48]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./mw/Residence_Haeundae.json'):
    with open(path, 'r',encoding='UTF-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str) 
        except(ValueError, TypeError):
            review_date = current_date 

        if review_date < date_boundary:
            continue
        
        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad) 
    
    return review_good_text, review_bad_text

good, bad = preprocess_reviews()

In [49]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

In [50]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 5문장 내로 요약해줘:""" 

In [51]:
from openai import OpenAI
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
reviews, _ = preprocess_reviews(path="./mw/Residence_Haeundae.json")
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

def summarize(reviews,prompt,temperature=0.0,model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion 

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

In [ ]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE,temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

In [14]:
summary_real_20241003 = '교통이 편리한 위치에 있으며, 청결한 상태를 유지하고 있어 많은 후기가 긍정적입니다. 객실은 넷플릭스 시청을 위한 기기 대여가 필요하지만 전반적으로 깔끔하고 편안한 분위기를 제공합니다. 직원들의 친절한 응대가 인상적이며, 조식과 디너 서비스 역시 우수하다는 평이 있습니다. 다만, 방음과 시설 노후화 관련 아쉬운 점이 언급되기도 했습니다. 종합적으로 접근성과 서비스가 우수한 조건을 갖춘 숙소입니다.'

In [ ]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt,b_cnt,draw_cnt=0,0,0 
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i]) 
        verdict_text = completion.choices[0].message.content 

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else: 
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt

wins, loses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20241003]*eval_count) 
print(f"Wins: {wins}, Loses: {loses}, Ties: {ties}")

## 고도화: 조건 명시 

In [34]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:43<00:00,  4.35s/it]

Wins: 0, Losses: 10, Ties: 0


## 고도화 2: 입력 데이터 품질 증가

In [48]:
def preprocess_reviews(path='./ms/Residence_Haeundae.json'):
    with open(path, 'r',encoding='UTF-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str) 
        except(ValueError, TypeError):
            review_date = current_date 

        if review_date < date_boundary:
            continue
        
        if len(review['review']) < 30:
            continue

        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad) 
    
    return review_good_text, review_bad_text

reviews, _ = preprocess_reviews(path="./mw/Residence_Haeundae.json")

In [37]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:49<00:00, 10.99s/it]

Wins: 1, Losses: 8, Ties: 1


##Few-Shot

In [46]:
reviews_1shot, _ = preprocess_reviews(path="./mw/Residence_Haeundae.json") 
summary_1shot = summarize(reviews_1shot, prompt, temperature=1.0, model = 'gpt-4o-2024-05-13').choices[0].message.content

In [49]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:56<00:00,  5.70s/it]

Wins: 7, Losses: 3, Ties: 0
